In [ ]:
from langchain_community.document_loaders import YoutubeLoader

url = "data"
loader = YoutubeLoader.from_youtube_url(url, add_video_info=False)
docs = loader.load()
doc = docs[0]

In [ ]:
from langchain_text_splitters import  RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_text(doc.page_content)

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import (SystemMessagePromptTemplate, 
                                    HumanMessagePromptTemplate,
                                    ChatPromptTemplate)
from langchain_core.output_parsers import StrOutputParser

system = SystemMessagePromptTemplate.from_template("""You are helpful AI assistant who creates summarization based on the provided context.""")
prompt = """Create a short summarization of the provided context!
            ### Context:
            {context}

            ### Answer:"""

llm = ChatOllama(model='data', base_url='data')
prompt = HumanMessagePromptTemplate.from_template(prompt)
messages = [system, prompt]
template = ChatPromptTemplate(messages)
qna_chain = template | llm | StrOutputParser()

In [ ]:
sums = []
for chunk in chunks:
    sum = qna_chain.invoke({'context':chunk})
    sums.append(sum)

In [ ]:
sums_join = ". ".join([sum.split('\n\n')[-1] for sum in sums])

In [ ]:
result = qna_chain.invoke({'context':sums_join})

In [ ]:
print(result)